## Part II: Predicting Hall of Fame Careers

In [59]:
# Connecting to SQLite Database
import pandas as pd
import sqlite3
conn = sqlite3.connect("lahman2016.sqlite")

In [60]:
queryMaster = 'select playerID,nameFirst,nameLast,bats,throws,debut,finalGame from Master;'
queryPitching = 'select * from Pitching;'
queryFielding = 'select * from Fielding;'
queryAwards = 'select playerID,awardID,yearID from AwardsPlayers;'
queryAllStar = 'select playerID, YearID from AllstarFull;'
queryHOF = 'select playerID,yearid from HallofFame where inducted == "Y" and category == "Player";'

In [61]:
batting_df = pd.read_csv('Batting.csv')

In [62]:
master = conn.execute(queryMaster).fetchall()
master_df = pd.DataFrame(master)
pitching = conn.execute(queryPitching).fetchall()
pitching_df = pd.DataFrame(pitching)
fielding = conn.execute(queryFielding).fetchall()
fielding_df = pd.DataFrame(fielding)
awards = conn.execute(queryAwards).fetchall()
awards_df = pd.DataFrame(awards)
allstar = conn.execute(queryAllStar).fetchall()
allstar_df = pd.DataFrame(allstar)
hof = conn.execute(queryHOF).fetchall()
hof_df = pd.DataFrame(hof)

In [63]:
master_cols = ['playerID','nameFirst','nameLast','bats','throws','debut','finalGame']
master_df.columns = master_cols

pitching_cols = ['playerID','yearID','stint','teamID','lgID','W','L','G','GS','CG','SHO',
'SV','IPOuts','H','ER','HR','BB','SO','BAOpp','ERA','IBB','WP','HBP','BK','BFP','GF','R','SH','SF','GIDP']
pitching_df.columns = pitching_cols

fielding_cols = ['playerID','yearID','stint','teamID','lgID','Pos','G','GS','InnOuts','PO','A','E','DP','PB','WP','SB','CS','ZR']
fielding_df.columns = fielding_cols

awards_cols = ['playerID','awardID','yearID']
awards_df.columns = awards_cols

allstar_cols = ['playerID','YearID']
allstar_df.columns = allstar_cols

hof_cols = ['playerID','yearid']
hof_df.columns = hof_cols

In [64]:
print(batting_df.head())

    playerID  yearID  stint teamID lgID   G   AB   R   H  2B  ...    RBI   SB  \
0  abercda01    1871      1    TRO  NaN   1    4   0   0   0  ...    0.0  0.0   
1   addybo01    1871      1    RC1  NaN  25  118  30  32   6  ...   13.0  8.0   
2  allisar01    1871      1    CL1  NaN  29  137  28  40   4  ...   19.0  3.0   
3  allisdo01    1871      1    WS3  NaN  27  133  28  44  10  ...   27.0  1.0   
4  ansonca01    1871      1    RC1  NaN  25  120  29  39  11  ...   16.0  6.0   

    CS  BB   SO  IBB  HBP  SH  SF  GIDP  
0  0.0   0  0.0  NaN  NaN NaN NaN   NaN  
1  1.0   4  0.0  NaN  NaN NaN NaN   NaN  
2  1.0   2  5.0  NaN  NaN NaN NaN   NaN  
3  1.0   0  2.0  NaN  NaN NaN NaN   NaN  
4  2.0   2  1.0  NaN  NaN NaN NaN   NaN  

[5 rows x 22 columns]


In [65]:
player_stats = {}
for i, row in batting_df.iterrows():
    playerID = row['playerID']
    G = row['G']
    AB = row['AB']
    R = row['R']
    H = row['H']
    twoB = row['2B']
    threeB = row['3B']
    HR = row['HR']
    RBI = row['RBI']
    SB = row['SB']
    CS = row['CS']
    BB = row['BB']
    SO = row['SO']
    IBB = row['IBB']
    HBP = row['HBP']
    SH = row['SH']
    SF = row['SF']
    GIDP = row['GIDP']
    if playerID in player_stats:
        player_stats[playerID]['G'] = player_stats[playerID]['G'] + G
        player_stats[playerID]['AB'] = player_stats[playerID]['AB'] + AB
        player_stats[playerID]['R'] = player_stats[playerID]['R'] + R
        player_stats[playerID]['H'] = player_stats[playerID]['H'] + H
        player_stats[playerID]['2B'] = player_stats[playerID]['2B'] + twoB
        player_stats[playerID]['3B'] = player_stats[playerID]['3B'] + threeB
        player_stats[playerID]['HR'] = player_stats[playerID]['HR'] + HR
        player_stats[playerID]['RBI'] = player_stats[playerID]['RBI'] + RBI
        player_stats[playerID]['SB'] = player_stats[playerID]['SB'] + SB
        player_stats[playerID]['CS'] = player_stats[playerID]['CS'] + CS
        player_stats[playerID]['BB'] = player_stats[playerID]['BB'] + BB
        player_stats[playerID]['SO'] = player_stats[playerID]['SO'] + SO
        player_stats[playerID]['IBB'] = player_stats[playerID]['IBB'] + IBB
        player_stats[playerID]['HBP'] = player_stats[playerID]['HBP'] + HBP
        player_stats[playerID]['SH'] = player_stats[playerID]['SH'] + SH
        player_stats[playerID]['GIDP'] = player_stats[playerID]['GIDP'] + GIDP
    else:
        player_stats[playerID] = {}
        player_stats[playerID]['G'] = G
        player_stats[playerID]['AB'] = AB
        player_stats[playerID]['R'] = R
        player_stats[playerID]['H'] = H
        player_stats[playerID]['2B'] = twoB
        player_stats[playerID]['3B'] = threeB
        player_stats[playerID]['HR'] = HR
        player_stats[playerID]['RBI'] = RBI
        player_stats[playerID]['SB'] = SB
        player_stats[playerID]['CS'] = CS
        player_stats[playerID]['BB'] = BB
        player_stats[playerID]['SO'] = SO
        player_stats[playerID]['IBB'] = IBB
        player_stats[playerID]['HBP'] = HBP
        player_stats[playerID]['SH'] = SH
        player_stats[playerID]['GIDP'] = GIDP        


In [66]:
pitcher_list = []
for i, row in pitching_df.iterrows():
    playerID = row['playerID']
    Wp = row['W']
    Lp = row['L']
    Gp = row['G']
    GSp = row['GS']
    CGp = row['CG']
    SHOp = row['SHO']
    SVp = row['SV']
    IPOutsp = row['IPOuts']
    Hp = row['H']
    ERp = row['ER']
    HRp = row['HR']
    BBp = row['BB']
    SOp = row['SO']
    IBBp = row['IBB']
    WPp = row['WP']
    HBPp = row['HBP']
    BKp = row['BK']
    BFPp = row['BFP']
    GFp = row['GF']
    Rp = row['R']
    SHp = row['SH']
    SFp = row['SF']
    GIDPp = row['GIDP']
    if playerID in player_stats and playerID in pitcher_list:
        player_stats[playerID]['Wp'] = player_stats[playerID]['Wp'] + Wp
        player_stats[playerID]['Lp'] = player_stats[playerID]['Lp'] + Lp
        player_stats[playerID]['Gp'] = player_stats[playerID]['Gp'] + Gp
        player_stats[playerID]['GSp'] = player_stats[playerID]['GSp'] + GSp
        player_stats[playerID]['CGp'] = player_stats[playerID]['CGp'] + CGp
        player_stats[playerID]['SHOp'] = player_stats[playerID]['SHOp'] + SHOp
        player_stats[playerID]['SVp'] = player_stats[playerID]['SVp'] + SVp
        player_stats[playerID]['IPOutsp'] = player_stats[playerID]['IPOutsp'] + IPOutsp
        player_stats[playerID]['Hp'] = player_stats[playerID]['Hp'] + Hp
        player_stats[playerID]['ERp'] = player_stats[playerID]['ERp'] + ERp
        player_stats[playerID]['HRp'] = player_stats[playerID]['HRp'] + HRp
        player_stats[playerID]['BBp'] = player_stats[playerID]['BBp'] + BBp
        player_stats[playerID]['SOp'] = player_stats[playerID]['SOp'] + SOp
        player_stats[playerID]['IBBp'] = player_stats[playerID]['IBBp'] + IBBp
        player_stats[playerID]['WPp'] = player_stats[playerID]['WPp'] + WPp
        player_stats[playerID]['HBPp'] = player_stats[playerID]['HBPp'] + HBPp
        player_stats[playerID]['BKp'] = player_stats[playerID]['BKp'] + BKp
        player_stats[playerID]['BFPp'] = player_stats[playerID]['BFPp'] + BFPp
        player_stats[playerID]['GFp'] = player_stats[playerID]['GFp'] + GFp
        player_stats[playerID]['Rp'] = player_stats[playerID]['Rp'] + Rp
        player_stats[playerID]['SHp'] = player_stats[playerID]['SHp'] + SHp
        player_stats[playerID]['SFp'] = player_stats[playerID]['SFp'] + SFp
        player_stats[playerID]['GIDPp'] = player_stats[playerID]['GIDPp'] + GIDPp
    else:
        pitcher_list.append(playerID)
        player_stats[playerID]['Wp'] = Wp
        player_stats[playerID]['Lp'] = Lp
        player_stats[playerID]['Gp'] = Gp
        player_stats[playerID]['GSp'] = GSp
        player_stats[playerID]['CGp'] = CGp
        player_stats[playerID]['SHOp'] = SHOp
        player_stats[playerID]['SVp'] = SVp
        player_stats[playerID]['IPOutsp'] = IPOutsp
        player_stats[playerID]['Hp'] = Hp
        player_stats[playerID]['ERp'] = ERp
        player_stats[playerID]['HRp'] = HRp
        player_stats[playerID]['BBp'] = BBp
        player_stats[playerID]['SOp'] = SOp
        player_stats[playerID]['IBBp'] = IBBp
        player_stats[playerID]['WPp'] = WPp
        player_stats[playerID]['HBPp'] = HBPp
        player_stats[playerID]['BKp'] = BKp
        player_stats[playerID]['BFPp'] = BFPp
        player_stats[playerID]['GFp'] = GFp
        player_stats[playerID]['Rp'] = Rp
        player_stats[playerID]['SHp'] = SHp
        player_stats[playerID]['SFp'] = SFp
        player_stats[playerID]['GIDPp'] = GIDPp

        

In [67]:
fielder_list = []
for i, row in fielding_df.iterrows():
    playerID = row['playerID']
    Gf = row['G']
    GSf = row['GS']
    InnOutsf = row['InnOuts']
    POf = row['PO']
    Af = row['A']
    Ef = row['E']
    DPf = row['DP']
    PBfc = row['PB']
    WPfc = row['WP']
    SBfc = row['SB']
    CSfc = row['CS']
    if playerID in player_stats and playerID in fielder_list:
        player_stats[playerID]['Gf'] = player_stats[playerID]['Gf'] + Gf
        player_stats[playerID]['GSf'] = player_stats[playerID]['GSf'] + GSf
        player_stats[playerID]['InnOutsf'] = player_stats[playerID]['InnOutsf'] + InnOutsf
        player_stats[playerID]['POf'] = player_stats[playerID]['POf'] + POf
        player_stats[playerID]['Af'] = player_stats[playerID]['Af'] + Af
        player_stats[playerID]['Ef'] = player_stats[playerID]['Ef'] + Ef
        player_stats[playerID]['DPf'] = player_stats[playerID]['DPf'] + DPf
        player_stats[playerID]['PBfc'] = player_stats[playerID]['PBfc'] + PBfc
        player_stats[playerID]['WPfc'] = player_stats[playerID]['WPfc'] + WPfc
        player_stats[playerID]['SBfc'] = player_stats[playerID]['SBfc'] + SBfc
        player_stats[playerID]['CSfc'] = player_stats[playerID]['CSfc'] + CSfc
    else:
        fielder_list.append(playerID)
        player_stats[playerID]['Gf'] = Gf
        player_stats[playerID]['GSf'] = GSf
        player_stats[playerID]['InnOutsf'] = InnOutsf
        player_stats[playerID]['POf'] = POf
        player_stats[playerID]['Af'] = Af
        player_stats[playerID]['Ef'] = Ef
        player_stats[playerID]['DPf'] = DPf
        player_stats[playerID]['PBfc'] = PBfc
        player_stats[playerID]['WPfc'] = WPfc
        player_stats[playerID]['SBfc'] = SBfc
        player_stats[playerID]['CSfc'] = CSfc

In [68]:
print(awards_df['awardID'].unique())

['Pitching Triple Crown' 'Triple Crown' 'Baseball Magazine All-Star'
 'Most Valuable Player' 'TSN All-Star' 'TSN Guide MVP'
 'TSN Major League Player of the Year' 'TSN Pitcher of the Year'
 'TSN Player of the Year' 'Rookie of the Year' 'Babe Ruth Award'
 'Lou Gehrig Memorial Award' 'World Series MVP' 'Cy Young Award'
 'Gold Glove' 'TSN Fireman of the Year' 'All-Star Game MVP' 'Hutch Award'
 'Roberto Clemente Award' 'Rolaids Relief Man Award' 'NLCS MVP' 'ALCS MVP'
 'Silver Slugger' 'Branch Rickey Award' 'Hank Aaron Award'
 'TSN Reliever of the Year' 'Comeback Player of the Year'
 'Outstanding DH Award' 'Reliever of the Year Award']


In [69]:
mvp_df = awards_df[awards_df['awardID'] == 'Most Valuable Player']
roy_df = awards_df[awards_df['awardID'] == 'Rookie of the Year']
cy_df = awards_df[awards_df['awardID'] == 'Cy Young Award']
gg_df = awards_df[awards_df['awardID'] == 'Gold Glove']
ss_df = awards_df[awards_df['awardID'] == 'Silver Slugger']

awards_list = [mvp_df,roy_df,cy_df,gg_df,ss_df]

In [70]:
print(awards_list[1].head())

       playerID             awardID  yearID
1796  robinja02  Rookie of the Year    1947
1845   darkal01  Rookie of the Year    1948
1899  sievero01  Rookie of the Year    1949
1900  newcodo01  Rookie of the Year    1949
1953  dropowa01  Rookie of the Year    1950


In [71]:
mvp_list = []
roy_list = []
cy_list = []
gg_list = []
ss_list = []
lists = [mvp_list,roy_list,cy_list,gg_list,ss_list]

In [72]:
for index, v in enumerate(awards_list):
    for i, row in v.iterrows():
        playerID = row['playerID']
        award = row['awardID']
        if playerID in player_stats and playerID in lists[index]:
            player_stats[playerID][award] += 1
        else:
            lists[index].append(playerID)
            player_stats[playerID][award] = 1

In [73]:
allstar_list = []
for i, row in allstar_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats and playerID in allstar_list:
        player_stats[playerID]['AS_games'] += 1
    else:
        allstar_list.append(playerID)
        player_stats[playerID]['AS_games'] = 1

In [74]:
for i, row in hof_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats:
        player_stats[playerID]['HoF'] = 1

### Add the master df into the dictionary here.  Then convert to data frame.

In [75]:
print(len(hof_df))
print(len(master_df))

250
19105


In [76]:
print(dict(list(player_stats.items())[0:2]))

{'waitked01': {'PBfc': nan, 'DPf': 886.0, 'Gf': 1069, 'H': 1214, 'SBfc': nan, 'InnOutsf': nan, 'G': 1140, 'Af': 716.0, 'GIDP': 75.0, 'SO': 204.0, 'GSf': nan, 'HBP': 11.0, 'CSfc': nan, 'SB': 28.0, 'AS_games': 2, '2B': 215, 'IBB': nan, 'CS': nan, 'SH': 40.0, 'R': 528, 'RBI': 373.0, 'POf': 9193, 'AB': 4254, '3B': 44, 'BB': 372, 'WPfc': nan, 'HR': 24, 'Ef': 73.0}, 'johnsbr01': {'PBfc': nan, 'DPf': 22.0, 'Gf': 428, 'H': 351, 'SBfc': nan, 'InnOutsf': 9903.0, 'G': 471, 'Af': 157.0, 'GIDP': 42.0, 'SO': 268.0, 'GSf': 368.0, 'HBP': 12.0, 'CSfc': nan, 'SB': 1.0, '2B': 60, 'IBB': 16.0, 'CS': 3.0, 'SH': 16.0, 'R': 132, 'RBI': 196.0, 'POf': 2629, 'AB': 1415, '3B': 6, 'BB': 80, 'WPfc': nan, 'HR': 49, 'Ef': 21.0}}


In [81]:
stats_df = pd.DataFrame.from_dict(player_stats, orient='index')

In [85]:
stats_df['playerID'] = stats_df.index 

In [88]:
print(stats_df['playerID'].head())
print(len(df))

aardsda01    aardsda01
aaronha01    aaronha01
aaronto01    aaronto01
aasedo01      aasedo01
abadan01      abadan01
Name: playerID, dtype: object
18915
